# Profiling for alchemlyb example
The purpose of this notebook is to understand performance bottleneck in a particular use case by profiling python functions.

## Benzene-in-water example for parsing

This example was described during the presentation and there is a similar test script using TI at https://github.com/alchemistry/alchemlyb/blob/master/src/alchemlyb/tests/test_ti_estimators.py

The highlighted point to investigate is about parsing multiple simulation files, and the specific function to run with the profiler is:

In [1]:
def gmx_benzene_coul_dHdl():
    dataset = alchemtest.gmx.load_benzene()

    dHdl = pd.concat([gmx.extract_dHdl(filename, T=300)
                      for filename in dataset['data']['Coulomb']])

    return dHdl

I need to spread the function out to have the execution time per line. For example, the load_benzene() is for five compressed xvg files like:

```

    $ ls ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/* -alh
    ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/0000:
    -rw------- 1 lee212 lee212 106K Feb  3 22:51 dhdl.xvg.bz2

    ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/0250:
    -rw------- 1 lee212 lee212 109K Feb  3 22:51 dhdl.xvg.bz2

    ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/0500:
    -rw------- 1 lee212 lee212 107K Feb  3 22:51 dhdl.xvg.bz2

    ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/0750:
    -rw------- 1 lee212 lee212 112K Feb  3 22:51 dhdl.xvg.bz2

    ~/venv/alchemlyb/lib/python2.7/site-packages/alchemtest-0.2.0rc1+6.g2f0f9a1-py2.7.egg/alchemtest/gmx/benzene/Coulomb/1000:
    -rw------- 1 lee212 lee212 110K Feb  3 22:51 dhdl.xvg.bz2
```

Five files with the size of about 100k bytes where the total size is around 1.5MB (300k * 5 files) when it's uncompressed.

The following function is same but with multiple lines:

In [ ]:
import pandas as pd
import alchemtest.gmx
from alchemlyb.parsing import gmx
from alchemlyb.estimators import TI

@profile
def gmx_benzene_coul_dHdl():
    dataset = alchemtest.gmx.load_benzene()
    d1 = gmx.extract_dHdl(bz['data']['Coulomb'][0], T=300)
    d2 = gmx.extract_dHdl(bz['data']['Coulomb'][1], T=300)
    d3 = gmx.extract_dHdl(bz['data']['Coulomb'][2], T=300)
    d4 = gmx.extract_dHdl(bz['data']['Coulomb'][3], T=300)
    d5 = gmx.extract_dHdl(bz['data']['Coulomb'][4], T=300)
    dHdl = pd.concat([d1, d2, d3, d4, d5])
    TI().fit(dHdl)

## Line profiler for gmx_benzene_coul_dHdl()

I used [line_profiler](https://github.com/rkern/line_profiler) and the result is followed:

```
Total time: 0.518848 s
File: profile_ti_estimator.py
Function: gmx_benzene_coul_dHdl at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     6                                           @profile
     7                                           def gmx_benzene_coul_dHdl():
     8         1      19848.0  19848.0      3.8      dataset = alchemtest.gmx.load_benzene()
     9         1     102725.0 102725.0     19.8      d1 = gmx.extract_dHdl(bz['data']['Coulomb'][0], T=300)
    10         1      92709.0  92709.0     17.9      d2 = gmx.extract_dHdl(bz['data']['Coulomb'][1], T=300)
    11         1      90751.0  90751.0     17.5      d3 = gmx.extract_dHdl(bz['data']['Coulomb'][2], T=300)
    12         1      91756.0  91756.0     17.7      d4 = gmx.extract_dHdl(bz['data']['Coulomb'][3], T=300)
    13         1      93399.0  93399.0     18.0      d5 = gmx.extract_dHdl(bz['data']['Coulomb'][4], T=300)
    14         1       6062.0   6062.0      1.2      dHdl = pd.concat([d1, d2, d3, d4, d5])
    15         1      21594.0  21594.0      4.2      TI().fit(dHdl)
```

pandas concat is inexpensive and let's look at the extract_dHdl() function where 90% of time is consumed.

Note that the Time value is microsecond in the above.

### extract_dHdl()  line-by-line profiling

The example here is parsing GROMACS xvg files by [parsing module, src/alchemlyb/parsing/gmx.py](https://github.com/alchemistry/alchemlyb/blob/master/src/alchemlyb/parsing/gmx.py#L112), and it indicates the file I/O is expensive where _extract_dataframe() is called (see line 131).

```
Timer unit: 1e-06 s

Total time: 0.322493 s
File: gmx.py
Function: extract_dHdl at line 111

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   111                                           @profile
   112                                           def extract_dHdl(xvg, T):
   113                                               """Return gradients `dH/dl` from a Hamiltonian differences XVG file.
   114                                           
   115                                               Parameters
   116                                               ----------
   117                                               xvg : str
   118                                                   Path to XVG file to extract data from.
   119                                           
   120                                               Returns
   121                                               -------
   122                                               dH/dl : Series
   123                                                   dH/dl as a function of time for this lambda window.
   124                                           
   125                                               """
   126         5         10.0      2.0      0.0      beta = 1/(k_b * T)
   127                                           
   128         5      53664.0  10732.8     16.6      state, lambdas, statevec = _extract_state(xvg)
   129                                           
   130                                               # extract a DataFrame from XVG data
   131         5     205188.0  41037.6     63.6      df = _extract_dataframe(xvg)
   132                                           
   133         5       1154.0    230.8      0.4      times = df[df.columns[0]]
   134                                           
   135                                               # want to grab only dH/dl columns
   136         5          5.0      1.0      0.0      dHcols = []
   137        10          8.0      0.8      0.0      for l in lambdas:
   138        45        274.0      6.1      0.1          dHcols.extend([col for col in df.columns if (l in col)])
   139                                           
   140         5       7797.0   1559.4      2.4      dHdl = df[dHcols]
   141                                           
   142                                               # make dimensionless
   143         5      14897.0   2979.4      4.6      dHdl = beta * dHdl
   144                                           
   145                                               # rename columns to not include the word 'lambda', since we use this for
   146                                               # index below
   147        10         45.0      4.5      0.0      cols = [l.split('-')[0] for l in lambdas]
   148                                           
   149         5        186.0     37.2      0.1      dHdl = pd.DataFrame(dHdl.values, columns=cols,
   150         5       2377.0    475.4      0.7                          index=pd.Float64Index(times.values, name='time'))
   151                                           
   152                                               # create columns for each lambda, indicating state each row sampled from
   153                                               # if state is None run as expanded ensemble data or REX
   154         5          5.0      1.0      0.0      if state is None:
   155                                                   # if thermodynamic state is specified map thermodynamic
   156                                                   # state data to lambda values, else (for REX)
   157                                                   # define state based on the legend
   158                                                   if 'Thermodynamic state' in df:
   159                                                       ts_index = df.columns.get_loc('Thermodynamic state')
   160                                                       thermo_state = df[df.columns[ts_index]]
   161                                                       for i, l in enumerate(lambdas):
   162                                                           v = []
   163                                                           for t in thermo_state:
   164                                                               v.append(statevec[int(t)][i])
   165                                                           dHdl[l] = v
   166                                                   else:
   167                                                       state_legend = _extract_legend(xvg)
   168                                                       for i, l in enumerate(state_legend):
   169                                                           dHdl[l] = state_legend[l]
   170                                               else:
   171        10         25.0      2.5      0.0          for i, l in enumerate(lambdas):
   172         5          5.0      1.0      0.0              try:
   173         5         13.0      2.6      0.0                  dHdl[l] = statevec[i]
   174         5          7.0      1.4      0.0              except TypeError:
   175         5       4646.0    929.2      1.4                  dHdl[l] = statevec
   176                                           
   177                                               # set up new multi-index
   178         5         17.0      3.4      0.0      newind = ['time'] + lambdas
   179         5      31846.0   6369.2      9.9      dHdl= dHdl.reset_index().set_index(newind)
   180                                           
   181         5        322.0     64.4      0.1      dHdl.name='dH/dl'
   182                                           
   183         5          2.0      0.4      0.0      return dHdl
```

### _extract_dataframe() line-by-line profiling

Another profiling for the internal function to read a file is followed:

```
Timer unit: 1e-06 s

Total time: 0.245378 s
File: gmx.py
Function: _extract_dataframe at line 226

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   226                                           @profile
   227                                           def _extract_dataframe(xvg):
   228                                               """Extract a DataFrame from XVG data.
   229
   230                                               """
   231         5       2505.0    501.0      1.0      with anyopen(xvg, 'r') as f:
   232         5          9.0      1.8      0.0          names = []
   233         5          4.0      0.8      0.0          rows = []
   234     20160      82187.0      4.1     33.5          for line in f:
   235     20155      12877.0      0.6      5.2              line = line.strip()
   236     20155      12271.0      0.6      5.0              if len(line) == 0:
   237                                                           continue
   238
   239     20155      12537.0      0.6      5.1              if "label" in line and "xaxis" in line:
   240         5         10.0      2.0      0.0                  xaxis = line.split('"')[-2]
   241
   242     20155      14119.0      0.7      5.8              if line.startswith("@ s") and "subtitle" not in line:
   243        35         51.0      1.5      0.0                  name = line.split("legend ")[-1].replace('"','').strip()
   244        35         23.0      0.7      0.0                  names.append(name)
   245
   246                                                       # should catch non-numeric lines so we don't proceed in parsing
   247                                                       # here
   248     20155      13943.0      0.7      5.7              if line.startswith(('#', '@')):
   249       150         82.0      0.5      0.0                  continue
   250
   251     20005      13550.0      0.7      5.5              if line.startswith('&'):  #pragma: no cover
   252                                                           raise NotImplementedError('{}: Multi-data not supported,'
   253                                                                                     'only simple NXY format.'.format(xvg))
   254                                                       # parse line as floats
   255     20005      50123.0      2.5     20.4              row = map(float, line.split())
   256     20005      13916.0      0.7      5.7              rows.append(row)
   257
   258         5          5.0      1.0      0.0      cols = [xaxis]
   259         5          9.0      1.8      0.0      cols.extend(names)
   260
   261         5      17157.0   3431.4      7.0      return pd.DataFrame(rows, columns=cols)
```

There might be some tuning points in this function for parsing headers and rendering floating values since gmx.extract_dHdl() function consumes most of the execution time here.

FYI, the content of a xvg file looks like:

```
# This file was created Tue Mar 21 13:50:08 2017
# Created by:
#                  :-) GROMACS - gmx energy, VERSION 5.1.4 (-:
# 
# Executable:   /nfs/packages/opt/Linux_x86_64/gromacs/5.1.4/cuda7.5/gnu-4.8/avx/bin/gmx
# Data prefix:  /nfs/packages/opt/Linux_x86_64/gromacs/5.1.4/cuda7.5/gnu-4.8/avx
# Command line:
#   gmx energy -odh /nfs/homes4/dldotson/Desktop/ikenney/Coulomb/0000/dhdl.xvg -s /nfs/homes4/dldotson/Desktop/ikenney/Coulomb/000
0/md.tpr -f /nfs/homes4/dldotson/Desktop/ikenney/Coulomb/0000/md.edr
# gmx energy is part of G R O M A C S:
#
# Gromacs Runs On Most of All Computer Systems
#
@    title "dH/d\xl\f{} and \xD\f{}H"
@    xaxis  label "Time (ps)"
@    yaxis  label "dH/d\xl\f{} and \xD\f{}H (kJ/mol [\xl\f{}]\S-1\N)"
@TYPE xy
@ subtitle "T = 300 (K) \xl\f{} state 0: fep-lambda = 0.0000"
@ view 0.15, 0.15, 0.75, 0.85
@ legend on
@ legend box on
@ legend loctype view
@ legend 0.78, 0.8
@ legend length 2
@ s0 legend "dH/d\xl\f{} fep-lambda = 0.0000"
@ s1 legend "\xD\f{}H \xl\f{} to 0.0000"
@ s2 legend "\xD\f{}H \xl\f{} to 0.2500"
@ s3 legend "\xD\f{}H \xl\f{} to 0.5000"
@ s4 legend "\xD\f{}H \xl\f{} to 0.7500"
@ s5 legend "\xD\f{}H \xl\f{} to 1.0000"
@ s6 legend "pV (kJ/mol)"
0.0000  33.399342 0.0000000 8.3498354 16.699671 25.049507 33.399342 0.77155721
10.0000  23.026176 0.0000000 5.7565441 11.513088 17.269632 23.026176 0.77036208
20.0000  13.227966 0.0000000 3.3069916 6.6139832 9.9209747 13.227966 0.75064653
30.0000  12.670597 0.0000000 3.1676493 6.3352985 9.5029478 12.670597 0.77252579
...
```

## Memory profiler for gmx_benzene_coul_dHdl()

Each xvg file is about 300kb size when it's uncompressed, and the memory consumption here is around 5MB for five 300kb xvg files.
```
Line #    Mem usage    Increment   Line Contents
================================================
     6   78.496 MiB   78.496 MiB   @profile
     7                             def gmx_benzene_coul_dHdl():
     8   78.508 MiB    0.012 MiB       dataset = alchemtest.gmx.load_benzene()
     9   83.871 MiB    5.363 MiB       d1 = gmx.extract_dHdl(dataset['data']['Coulomb'][0], T=300)
    10   87.473 MiB    3.602 MiB       d2 = gmx.extract_dHdl(dataset['data']['Coulomb'][1], T=300)
    11   87.484 MiB    0.012 MiB       d3 = gmx.extract_dHdl(dataset['data']['Coulomb'][2], T=300)
    12   87.496 MiB    0.012 MiB       d4 = gmx.extract_dHdl(dataset['data']['Coulomb'][3], T=300)
    13   87.496 MiB    0.000 MiB       d5 = gmx.extract_dHdl(dataset['data']['Coulomb'][4], T=300)
    14   87.512 MiB    0.016 MiB       dHdl = pd.concat([d1, d2, d3, d4, d5])
    15   87.684 MiB    0.172 MiB       TI().fit(dHdl)
```

## Preprosessing - statistical_inefficiency()

Profiling for preprocessing is also available, and this is just to explore other optimization opportunities.
The code snippet below is based on the test script here: https://github.com/alchemistry/alchemlyb/blob/master/src/alchemlyb/tests/test_preprocessing.py

```
Timer unit: 1e-06 s

Total time: 0.13373 s
File: alchemlyb_preprocessing.py
Function: main at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    14                                           @profile
    15                                           def main():
    16         1      68344.0  68344.0     51.1      d = gmx_benzene_dHdl()
    17         1      11295.0  11295.0      8.4      slicing(d, lower=1000, upper=34000, step=5)
    18         1      22904.0  22904.0     17.1      statistical_inefficiency(d, series=d.iloc[:, 0], conservative=True)
    19         1      31187.0  31187.0     23.3      statistical_inefficiency(d, series=d.iloc[:, 0], conservative=False)
```    

### statistical_inefficiency() line-by-line profiling
[This function](https://github.com/alchemistry/alchemlyb/blob/master/src/alchemlyb/preprocessing/subsampling.py) uses pymbar and the profiling results is:

```
Total time: 0.054 s
File: subsampling.py
Function: statistical_inefficiency at line 55

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    55                                           @profile
    56                                           def statistical_inefficiency(df, series=None, lower=None, upper=None, step=None,
    57                                                                        conservative=True):
   ... (skipped description) ...
   118         2      12560.0   6280.0     23.3      if _check_multiple_times(df):
   119                                                   raise KeyError("Duplicate time values found; statistical inefficiency "
   120                                                                  "only works on a single, contiguous, "
   121                                                                  "and sorted timeseries.")
   122                                           
   123         2       3767.0   1883.5      7.0      if not _check_sorted(df):
   124                                                   raise KeyError("Statistical inefficiency only works as expected if "
   125                                                                  "values are sorted by time, increasing.")
   126                                           
   127         2          3.0      1.5      0.0      if series is not None:
   128         2      14838.0   7419.0     27.5          series = slicing(series, lower=lower, upper=upper, step=step)
   129                                           
   130         2         22.0     11.0      0.0          if (len(series) != len(df) or
   131         2      11881.0   5940.5     22.0              not all(series.reset_index()['time'] == df.reset_index()['time'])):
   132                                                       raise ValueError("series and data must be sampled at the same times")
   133                                           
   134                                                   # calculate statistical inefficiency of series (could use fft=True but needs test)
   135         2        760.0    380.0      1.4          statinef  = statisticalInefficiency(series, fast=False)
   136                                           
   137                                                   # use the subsampleCorrelatedData function to get the subsample index
   138         2          2.0      1.0      0.0          indices = subsampleCorrelatedData(series, g=statinef,
   139         2       8703.0   4351.5     16.1                                            conservative=conservative)
   140         2       1462.0    731.0      2.7          df = df.iloc[indices]
   141                                               else:
   142                                                   df = slicing(df, lower=lower, upper=upper, step=step)
   143                                           
   144         2          2.0      1.0      0.0      return df
```

# Conclusion

If we think this is useful, we may run additional experiments for scaling tests e.g. with a large number of xvg files and different file sizes.